In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.test.is_built_with_xla()

True

In [1]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.image import resize

from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers

from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


tf.random.set_seed(13)
np.random.seed(13)

In [2]:
LEARNING_RATE = 1e-4
EPOCHS = 20
BATCH_SIZE = 32

In [3]:
images_path = 'dataset/chandrikadeb'

In [4]:
input_shape = (224, 224, 3)

In [5]:
def prepare_dataset(dataset_path, input_shape, class_mapping):

    dir_classes = os.listdir(dataset_path)
    data = []
    labels = []
    exts = ('.png', '.jpg', '.jpeg')


    for class_directory in dir_classes:
        dir_path = os.path.join(dataset_path, class_directory)
        imgs_filenames = [filename for filename in os.listdir(dir_path) if filename.endswith(exts)]
        imgs_full_paths = [os.path.join(dir_path, img_filename) for img_filename in imgs_filenames]
        for img_path in imgs_full_paths:
            img = img_to_array(load_img(img_path))
            img = preprocess_image(img, input_shape)
            data.append(img)
            labels.append(class_mapping[class_directory])

    return data, labels


In [6]:
def preprocess_image(image, input_shape):
    image = preprocess_input(image)
    image = resize(image, input_shape)
    return image

In [7]:
# data, labels = prepare_dataset(images_path, input_shape[:-1])

In [8]:
def to_one_hot(labels):
    onehot = OneHotEncoder()
    labels = np.array(labels).reshape(-1, 1)
    labels_one_hot = onehot.fit_transform(labels).toarray()
    
    return labels_one_hot

In [9]:
# labels_one_hot = to_one_hot(labels)

In [10]:
from tensorflow.keras import Model

def construct_classification_model(input_shape):
    mobilenet = MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in mobilenet.layers:
        layer.trainable = False
    mobilenet_out = mobilenet.output
    model = layers.GlobalAveragePooling2D()(mobilenet_out)
    model = layers.Dense(512, activation='relu')(model)
    model = layers.Dropout(0.5)(model)
    model = layers.Dense(2, activation='softmax')(model)

    network = Model(inputs=[mobilenet.input], outputs=[model])

    return network


In [11]:
model = construct_classification_model(input_shape)

In [12]:
img_data_gen = ImageDataGenerator(
    rescale=1./255., samplewise_center=True,
    samplewise_std_normalization = True,
    rotation_range=30, zoom_range=0.10,
    width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.15, horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [13]:
train_generator = img_data_gen.flow_from_directory(
    images_path, 
    subset='training',
    batch_size=BATCH_SIZE
)

validation_generator = img_data_gen.flow_from_directory(
    images_path,
    subset='validation',
    batch_size=BATCH_SIZE
)

Found 3067 images belonging to 2 classes.
Found 766 images belonging to 2 classes.


In [14]:
train_generator.class_indices

{'with_mask': 0, 'without_mask': 1}

In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
history = model.fit(train_generator, 
                    steps_per_epoch=len(train_generator), validation_data=validation_generator,
                    validation_steps = len(validation_generator), epochs=20)

Epoch 1/20
39/96 [===========>..................] - ETA: 24s - loss: 0.2766 - accuracy: 0.8974

C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


96/96 [==============================] - 56s 581ms/step - loss: 0.1752 - accuracy: 0.9371 - val_loss: 0.1244 - val_accuracy: 0.9530
Epoch 2/20
96/96 [==============================] - 52s 538ms/step - loss: 0.0841 - accuracy: 0.9707 - val_loss: 0.0288 - val_accuracy: 0.9922
Epoch 3/20
96/96 [==============================] - 50s 518ms/step - loss: 0.0710 - accuracy: 0.9739 - val_loss: 0.0337 - val_accuracy: 0.9883
Epoch 4/20
96/96 [==============================] - 50s 522ms/step - loss: 0.0612 - accuracy: 0.9742 - val_loss: 0.0251 - val_accuracy: 0.9896
Epoch 5/20
96/96 [==============================] - 50s 520ms/step - loss: 0.0483 - accuracy: 0.9801 - val_loss: 0.0439 - val_accuracy: 0.9856
Epoch 6/20
96/96 [==============================] - 50s 521ms/step - loss: 0.0527 - accuracy: 0.9798 - val_loss: 0.0233 - val_accuracy: 0.9909
Epoch 7/20
96/96 [==============================] - 50s 522ms/step - loss: 0.0405 - accuracy: 0.9843 - val_loss: 0.0289 - val_accuracy: 0.9909
Epoch 8/20

In [28]:
from tensorflow.keras.models import save_model

In [29]:
model.save('models/facemask-classification')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/facemask-classification\assets


In [17]:
loss, accuracy, val_loss, val_accuracy = history.history.values()

In [18]:
test_dataset_path = 'dataset/test'

In [19]:
train_generator.class_indices

{'with_mask': 0, 'without_mask': 1}

In [20]:
test_data, test_labels = prepare_dataset(test_dataset_path, input_shape[:-1], train_generator.class_indices)

In [21]:
test_labels_one_hot = to_one_hot(test_labels)

In [22]:
test_dataset = tf.data.Dataset.from_tensors(test_data)

In [23]:
numpy_list = [item.numpy() for item in test_data]

In [24]:
test_dataset_numpy = np.asarray(numpy_list)

In [25]:
test_dataset_numpy.shape

(40, 224, 224, 3)

In [26]:
test_dataset

<TensorDataset shapes: (40, 224, 224, 3), types: tf.float32>

In [27]:
predictions = model.evaluate(test_dataset_numpy, test_labels_one_hot)

2/2 [==============================] - 0s 140ms/step - loss: 1.4275e-05 - accuracy: 1.0000


In [ ]:
from tensorflow.keras.